In [13]:

import google.generativeai as genai
from dotenv import load_dotenv
import os
import json
import time
from tqdm import tqdm

load_dotenv()

genai.configure(api_key=os.environ["GEMINI_API_KEY"])
# The Gemini 1.5 models are versatile and work with both text-only and multimodal prompts
model = genai.GenerativeModel('gemini-1.5-flash')

In [14]:
DATA_URL = r"C:\Users\hoang\OneDrive\Desktop\nghich_prj\Hackathon\Hack-CMC-Heineken\data\FULL [Heineken Vietnam] Developer Resources"

In [15]:
PROMPT_VI = """

"""

PROMPT_EN = """
You are given an image related to Heineken beer products and various activities to enhance their service quality in customer experience.
The images may depict grocery stores with Heineken products, street activities by Heineken staff, bars, pubs, restaurants, special events like gameshows, and more.
Analyze each image and provide the following information:

- context: The place context in the image. For examples: bar, restaurant, grocery store, supermarket, small pub,... Just return me the place, not explain or say any un need words
- activity: list of short keyword describe human activities happend in image
- emotion: list of the main emotion of people in the image 
- effectiveness: If there are activities to enhance the user experience for example event, enjoy product..., i want to know how success, efficiency it be or whether it need to be improved, how to improve in short

Ensure the values are short, informative, and useful for further statistical analysis. If any of the criteria do not apply to the image, return null for that key.
Format the response in JSON

**Example response:**
{
  "context": "bar"
  "activity": ["drinking", "eating"],
  "emotion": ["happy", "enjoyable"],
  "effectiveness": null,
}
{
  "context": "street"
  "activity": ["beer promotion", "play game"],
  "emotion": ["happy", "interested"],
  "effectiveness": "highly effective",
}
{
  "context": "grocery store",
  "activity": null,
  "emotion": null,
  "effectiveness": null
}

"""

In [16]:
def get_response(img_path):
    sample_file = genai.upload_file(path=img_path,
                            display_name="Image")

    response = model.generate_content([sample_file, PROMPT_EN])
    return response.text

In [17]:
results = []
with open("./response_activities_emo_en.json", "r") as f:
    results = json.load(f)
    successed = [item['path'] for item in results]

In [18]:
failed = []

for img_name in tqdm(os.listdir(DATA_URL)):
    if img_name in successed:
        print("SKIP")
        continue
    try:
        fpath = os.path.join(DATA_URL, img_name)
        response  = get_response(fpath)
        results.append({
            "path": img_name,
            "response": response
        })
        with open("./response_activities_emo_en.json", "w") as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        time.sleep(3)
    except Exception as e:
        failed.append({
            "path": img_name,
            "reason": str(e)
        })
        with open("./failed.json", "w") as f:
            json.dump(failed, f, ensure_ascii=False, indent=2)
    

100%|██████████| 1484/1484 [11:20:34<00:00, 27.52s/it]       
